In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.bn1 = nn.BatchNorm2d(8)
        self.bn2 = nn.BatchNorm2d(16)
        self.dropout = nn.Dropout(0.05)
        # --- Convolution layers ---
        # Conv1: 1 -> 8, 3X3, padding=1
        # self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.05),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.05),
            nn.MaxPool2d(2,2)
        )

        # Conv2: 8 -> 12, 5x5, no padding
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=1, padding=1), #13X13
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.05),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3), #11X11
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.05),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3), #9X9  Rf= 16
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.05),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3), #7X7  Rf= 18
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.05),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=10, kernel_size=3), #13X13
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.05),
            nn.AvgPool2d(5,5)
        )

    def forward(self, x):
        # Conv + Pool
        x = self.conv1(x)
        x =self.conv2(x) # Output: 8 x 13 x 13
        x = self.conv3(x)   # Output: 16 x 4 x 4

        # Flatten
        x = x.view(-1, 10)

        return F.log_softmax(x, dim=1)


In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
           Dropout-4            [-1, 8, 28, 28]               0
            Conv2d-5           [-1, 16, 26, 26]           1,168
              ReLU-6           [-1, 16, 26, 26]               0
       BatchNorm2d-7           [-1, 16, 26, 26]              32
           Dropout-8           [-1, 16, 26, 26]               0
         MaxPool2d-9           [-1, 16, 13, 13]               0
           Conv2d-10            [-1, 8, 15, 15]             136
             ReLU-11            [-1, 8, 15, 15]               0
      BatchNorm2d-12            [-1, 8, 15, 15]              16
          Dropout-13            [-1, 8, 15, 15]               0
           Conv2d-14           [-1, 16,

In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform = transforms.Compose([
                        transforms.RandomRotation(10),         # small rotation (-10° to +10°)
                        transforms.RandomAffine(0, translate=(0.1, 0.1)),  # small shifts
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])
                    ),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

100%|██████████| 9.91M/9.91M [00:00<00:00, 37.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.02MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.31MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 11.8MB/s]


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target, label_smoothing=0.1)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.015, momentum=0.9, weight_decay=1e-4)
# Scheduler - reduces LR by factor 0.1 after every 5 epochs
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
for epoch in range(0, 20):
    print('\n Epoch:',epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()


 Epoch: 0


loss=0.6420286297798157 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 20.04it/s]



Test set: Average loss: 0.1335, Accuracy: 9824/10000 (98.24%)


 Epoch: 1


loss=0.6353971362113953 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.82it/s]



Test set: Average loss: 0.0958, Accuracy: 9904/10000 (99.04%)


 Epoch: 2


...


Test set: Average loss: 0.0905, Accuracy: 9953/10000 (99.53%)


 Epoch: 18


loss=0.5346418619155884 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.79it/s]



Test set: Average loss: 0.0877, Accuracy: 9954/10000 (99.54%)


 Epoch: 19


loss=0.5498578548431396 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.58it/s]



Test set: Average loss: 0.0902, Accuracy: 9954/10000 (99.54%)

